In [1]:
using Pkg
Pkg.activate("/home/zhenan/projects/def-mpf/zhenan/julia/dev/AtomicOpt")

  Activating environment at `~/projects/def-mpf/zhenan/julia/dev/AtomicOpt/Project.toml`


In [2]:
using Revise
using AtomicOpt
using LinearAlgebra
using SparseArrays
using Printf
using Arpack
using Plots

## Load data

In [3]:
function load_data(data::String)
    if data == "temperature"
        file = open("./temp2020.csv", "r")
        I = Vector{Int64}(); J = Vector{Int64}(); V = Vector{Float64}()
        numRows=0; RowDict = Dict{String, Int64}();
        numCols=0; ColDict = Dict{String, Int64}();
        while !eof(file)
            line = readline(file)
            info = split(line, ",")
            # get value
            v = parse(Float64, info[3])
            if v!= 0
                push!(V, v)
                # get row index
                if !haskey(RowDict, info[1])
                    numRows += 1
                    RowDict[info[1]] = numRows
                end
                i = RowDict[info[1]]
                push!(I, i)
                # get column index
                if !haskey(ColDict, info[2])
                    numCols += 1
                    ColDict[info[2]] = numCols
                end
                j = ColDict[info[2]]
                push!(J, j)
            end    
        end
        close(file)
        X = sparse(I,J,V,numRows,numCols)
        X ./= norm(X)
        # low rank approximation
        m, n = size(X)
        r = 5
        U, S, V = svds(X, nsv=r)[1]
        Xopt = U*Diagonal(S)*V'
        @show norm(Xopt - X) / norm(X)
        # mask 
        p = 0.5
        mask = sprand(Bool, m, n, p)
        mask = convert(SparseMatrixCSC{Float64, Int64}, mask)
        I,J,V = findnz(mask)
        # measurement
        b = Vector{Float64}()
        bopt = Vector{Float64}()
        l = length(I)
        for t in 1:l
            i, j = I[t], J[t]
            push!(b, X[i,j])
            push!(bopt, Xopt[i,j])
        end
        # optimal parameters
        τ = sum(S)
        z = b - bopt
        α = norm(z)^2/2
        Z = sparse(I, J, z)
        λ = dot(Z, Xopt)/τ
        return m, n, r, X, mask, b, α, τ, λ
    elseif data == "random"
        m, n, r = 100, 100, 3
        X = rand(m, n)
        U, S, V = svd(X)
        X = U[:,1:r] * Diagonal( S[1:r] ) * V[:,1:r]'
        # mask 
        p = 0.5
        mask = sprand(Bool, m, n, p)
        mask = convert(SparseMatrixCSC{Float64, Int64}, mask)
        I,J,V = findnz(mask)
        # measurement 
        b = Vector{Float64}()
        l = length(I)
        for t in 1:l
            i, j = I[t], J[t]
            push!(b, X[i,j])
        end
        η = rand(l)
        ϵ = 1; η .*= (ϵ/norm(η))
        η = zeros(l)
        b .+= η
        # optimal parameters
        τ = sum(S[1:r])
        α = norm(η)^2/2
        Z = sparse(I, J, η)
        λ = dot(Z, X)/τ
        return m, n, r, X, mask, b, α, τ, λ
    else
        println("invalid data")
        return
    end
end

load_data (generic function with 1 method)

In [4]:
m, n, r, Xopt, mask, b, αopt, τopt, λopt = load_data("temperature");

norm(Xopt - X) / norm(X) = 0.24679324943862013


In [5]:
τopt

1.5381942604878027

## Solve matrix completion problem

In [9]:
Mop = MaskOP(mask)
A = NucBall(m, n, r)
dual_gaps = Vector{Float64}()
pr_infeas = Vector{Float64}()
sol = level_set(Mop, b, A; 
    α=αopt, tol=1e-2, maxIts=1000, τmax=τopt, dual_gaps=dual_gaps, infeas=pr_infeas);


  -------------------------------------------------------------------------
  Polar Level Set Method
  -------------------------------------------------------------------------
  number of variables    2488068         number of constraints 1244460
  feasibility tolerance  2.50e-05         α                    1.52e-02
  max iterations            1000 
  -------------------------------------------------------------------------
  Major      Minor        u-α        ℓ-α        gap          τ         infeas-α  Subproblem
      1          2   7.32e-02   4.47e-02   2.85e-02   5.44e-01       1.32e-02   suboptimal
      2          2   3.62e-02   3.49e-02   1.31e-03   7.87e-01       1.22e-02   suboptimal
      3          2   1.18e-02   6.96e-03   4.84e-03   1.07e+00       1.22e-02   suboptimal
      4          4   6.37e-03   1.01e-03   5.36e-03   1.19e+00       1.22e-02   suboptimal
      5          3   5.46e-03   3.01e-03   2.45e-03   1.21e+00       1.22e-02   suboptimal
      6          6   3

## Report relative difference

In [20]:
x = constructPrimal(sol)
X = reshape(x, m, n)
@printf "The relative difference between Xopt and X: %e \n" norm(Xopt - X)/norm(Xopt)

The relative difference between Xopt and X: 3.103530e-01 


## Print dual gap and infeas

In [24]:
gaps = [5.44e-01, 7.87e-01, 1.07e+00, 1.19e+00, 1.21e+00, 1.30e+00, 1.30e+00, 1.30e+00, 1.33e+00, 1.35e+00, 1.36e+00, 
    1.37e+00, 1.39e+00, 1.39e+00, 1.39e+00, 1.40e+00, 1.40e+00, 1.40e+00, 1.40e+00, 1.40e+00, 1.41e+00, 1.41e+00,
    1.41e+00, 1.41e+00]
gaps = [τopt - g for g in gaps]
gr(size=(500,400))
rounds = collect(1:length(dual_gaps))
p = plot(rounds, gaps, label="duality gap", yaxis=:log, line=2)
plot!(rounds, pr_infeas, label="recovery error", yaxis=:log, line=2)
xaxis!("iteration", xtickfontsize=10)
yaxis!("gap", ytickfontsize=10)
savefig(p,"./matrix_completion.pdf")

In [23]:
size(X)

(6798, 366)